In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Downloading files

In [ ]:
# !wget --header="Host: www.ncbi.nlm.nih.gov" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-GB,en;q=0.9,en-US;q=0.8,bn;q=0.7" --header="Referer: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE214611" --header="Cookie: _gid=GA1.2.843222926.1683780721; ncbi_sid=0C4234AB45C74711_0000SID; _ga=GA1.1.1244755766.1683780721; _ga_DP2X732JSX=GS1.1.1683780720.1.1.1683782404.0.0.0; _gat_ncbiSg=1; ncbi_pinger=N4IgDgTgpgbg+mAFgSwCYgFwgAwGEAsATAMwDsAjAGz4Csup+pl52rb7H5IANCAMYAbZHwDWAOygAPAC6ZQxAJyYQVUkwAc2HiHxasqhfirb8+ZYOHips3rWVdblZQDMAhgIDOUE6WXUttur25PiUAHSUkZTqpNjk5BTqQbZKWNo0eiCI0tJgHhgA9AUA7qVhYnwARsjlAgC25ciIYQDmAPYwBS1QbQWobcViAm2uqAUA/K58fAC8AOIAygCihCHM5ABkzm0Qda7SM87IAt68NFxpZ4TKABTZuQCUIAC+vACuQyOoEjJyIMTEFzuLzaYhmLDSCBvU7/IKXf6pFSUNSUQwmTIGIhKWzgkAfYajH42HQ0ZTXM5OLA0VHpXxYAIgZjKTSA178Np1OptMREv7XLDdNqg5QmFxvAQCdIi3iETJ4IhkKi0eiMZgcdXsBwgVbKQUYAByAHl9UsMM5xQIDcaltpCPyQKViuUqjUhg0xE1Wh1bYjyApZaCMdhNKCLip1Phkv97SwaIDbID6WFiOowuSQHH7CR0zRcdTfM9nkA" --header="Connection: keep-alive" "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE214611&format=file" -c -O '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW.tar'

In [ ]:
# !tar -xvf '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW.tar' -C '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/'

In [ ]:
# !cd '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/' && for f in *.tar.gz; do tar -xvzf "$f" -C "$(dirname "$f")"; done

In [ ]:
# !unzip '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping.zip' -d '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW'

## Working with the data

In [2]:
!pip install anndata==0.10.3 scanpy==1.9.6 contrastive contrastive-vi

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
Requested pytorch-lightning<1.6,>=1.5 from https://files.pythonhosted.org/packages/18/f1/f59b307f75db1886c96e396eec878501510677394868680b8d2b8b58c47c/pytorch_lightning-1.5.10-py3-none-any.whl (from scvi-tools==0.16.1->contrastive-vi) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    torch (>=1.7.*)
           ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested pytorch-lightning<1.6,>=1.5 from https://files.pythonhosted.org/packages/64/e0/6f2cece92949328d01bf804a70d86858673ffd41772b27d9f60e5822404f/pytorch_lightning-1.5.9-py3-none-any.whl (from scvi-tools==0.16.1->contrastive-vi) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    torch (>=1.7.*)
           ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested pytorch-lightning<1.6,>=1.5 from https://files.pythonhosted.or

In [1]:
import scanpy as sc
import pandas as pd

In [2]:
ROOT_PATH = '/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/data/Infracted Heart Border Zone - GSE214611_RAW'

In [3]:
metadata_sc = pd.read_csv(f'{ROOT_PATH}/Single-Cell-Spatial-Transcriptomics-for-Border-zone/sn_wc_object_integrated@meta.data.csv', index_col=0)
metadata_sc_t = metadata_sc.rename(index=lambda x: x.split('_')[0])
metadata_sc_t['obs_names'] = metadata_sc_t.index

In [4]:
def get_anndata(mtx_path_id, metadata_orig_id):
    PATH = f'{ROOT_PATH}/{mtx_path_id}'
    ad = sc.read_10x_mtx(PATH)
    ad.obs = ad.obs.rename(index=lambda x: x.split('-')[0])
    A = metadata_sc_t[metadata_sc_t['orig.ident'] == metadata_orig_id]

    ad.obs = ad.obs.merge(A, left_index=True, right_index=True, how='left')

    return ad

In [5]:
def pp_anndata(ad, n_top_genes=None, plot=False):
    ad.var_names_make_unique()

    sc.pp.filter_cells(ad, min_genes=200)
    sc.pp.filter_genes(ad, min_cells=3)

    ad.var['mt'] = ad.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(ad, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    if plot:
        sc.pl.violin(ad, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
                jitter=0.4, multi_panel=True)
        sc.pl.scatter(ad, x='total_counts', y='pct_counts_mt')
        sc.pl.scatter(ad, x='total_counts', y='n_genes_by_counts')

    ad = ad[ad.obs.pct_counts_mt <= 5, :]
    ad.layers['counts'] = ad.X.copy()
    sc.pp.normalize_total(ad, target_sum=1e4)
    sc.pp.log1p(ad)

    if n_top_genes is not None:
        sc.pp.highly_variable_genes(ad, n_top_genes=n_top_genes)
        ad.raw = ad
        ad = ad[:, ad.var.highly_variable]

    if plot:
        sc.pl.highly_variable_genes(ad)

    return ad

## Prepare training and testing dataset

In [6]:
def get_mtx_path_id_metadata_orig_id(problem_key):
    exp_meta_keys = {
        'ref_0_que_1hr': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn1hr_1', 'sn1_1'),
                ('sn1hr_2', 'sn1_2'),
            ]
        },
        'ref_0_que_4hr': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn4hr', '4HR'),
            ]
        },
        'ref_0_que_1': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd1_1', 'D1_MI'),
                ('snd1_2', 'D1_MI_Hrt_nuclei'),
                ('snd1_3', 'D1_IR30_Hrt_nuclei'),
                ('snd1_4', 'WT_IR30_D1_2'),
                ('snd1_5', 'WT_IR30_D1_3'),
            ]
        },
        'ref_0_que_3': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd3_1', 'WT_IR30_D3'),
                ('snd3_2', 'D3F_M_rep1'),
                ('snd3_3', 'D3F_M_rep2'),
            ]
        },
        'ref_0_que_7': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd7_1', 'snD7_1'),
                ('snd7_2', 'snD7_1'),
                ('snd7_3', 'D5_D7'),
            ]
        },
        'ref_0_que_1-7': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn1hr_1', 'sn1_1'),
                ('sn1hr_2', 'sn1_2'),
                ('sn4hr', '4HR'),
                ('snd1_1', 'D1_MI'),
                ('snd1_2', 'D1_MI_Hrt_nuclei'),
                ('snd1_3', 'D1_IR30_Hrt_nuclei'),
                ('snd1_4', 'WT_IR30_D1_2'),
                ('snd1_5', 'WT_IR30_D1_3'),
                ('snd3_1', 'WT_IR30_D3'),
                ('snd3_2', 'D3F_M_rep1'),
                ('snd3_3', 'D3F_M_rep2'),
                ('snd7_1', 'snD7_1'),
                ('snd7_2', 'snD7_1'),
                ('snd7_3', 'D5_D7'),
            ]
        }
    }

    return exp_meta_keys[problem_key]

In [7]:
import anndata as AD

imp_celltypes = ['Ankrd1', 'Xirp2', 'Myh6']


def get_ad(PROBLEM_KEY):

    exp_meta_keys = get_mtx_path_id_metadata_orig_id(PROBLEM_KEY)

    adatas = {}

    for key, item in exp_meta_keys.items():
        for i, (mtx_path_id, metadata_orig_id) in enumerate(item):
            ad = pp_anndata(get_anndata(mtx_path_id, metadata_orig_id))
            ad = ad[ad.obs['final_cluster'].isin(imp_celltypes)]
            dataset_id = f"{key}_{i+1}"
            ad.obs['sample'] = dataset_id
            ad.obs['dataset'] = key
            adatas[f"{key}_{i+1}"] = ad

    ad = AD.concat(adatas, label='batch_key')
    ad.obs_names_make_unique()

    cluster_to_zone = {
        "Ankrd1": "BZ1",
        "Xirp2": "BZ2",
        "Myh6": "RZ",
    }

    ad.obs["zone"] = ad.obs["final_cluster"].map(cluster_to_zone)

    return ad

In [8]:
healthy_celltypes = ['Myh6']
disease_celltypes = ['Ankrd1', 'Xirp2']
imp_celltypes = disease_celltypes + healthy_celltypes

In [9]:
import time


with open('metrics.csv', 'w') as f:
    print('PROBLEM_KEY', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'aupr', 'time_taken',  sep=',', file=f, flush=True)


    for PROBLEM_KEY in ['ref_0_que_7',  'ref_0_que_1-7']:
        ad = get_ad(PROBLEM_KEY)

        reference_samples = list(ad.obs[ad.obs['dataset'] == 'reference']['sample'].unique())
        query_samples = list(ad.obs[ad.obs['dataset'] == 'query']['sample'].unique())


        for i in range(5):
            print('RUN: ', i)
            seed = i * 100

            start_time = time.perf_counter()

            ## ContrastiveVI start

            from contrastive_vi.model import ContrastiveVI
            from pytorch_lightning.utilities.seed import seed_everything
            import scanpy as sc
            seed_everything(seed)

            ContrastiveVI.setup_anndata(ad, layer="counts")

            target_ad = ad[ad.obs["dataset"].isin(['query'])].copy()
            background_ad = ad[ad.obs["dataset"].isin(['reference'])].copy()

            print(target_ad)
            print(background_ad)

            model = ContrastiveVI(ad)

            import numpy as np

            background_indices = np.where(ad.obs["dataset"].isin(['reference']))[0]
            target_indices = np.where(ad.obs["dataset"].isin(['query']))[0]

            model.train(
                check_val_every_n_epoch=1,
                train_size=0.8,
                background_indices=background_indices,
                target_indices=target_indices,
                early_stopping=True,
                max_epochs=500,
            )

            from anndata import AnnData

            salient_ad = AnnData(
                X=model.get_latent_representation(target_ad, representation_kind='salient'),
                obs=target_ad.obs
            )

            from sklearn.metrics import adjusted_rand_score
            from sklearn.cluster import KMeans

            true_labels = salient_ad.obs['final_cluster'].isin(disease_celltypes).to_numpy().astype(int)
            kmeans = KMeans(n_clusters=2)
            kmeans.fit(salient_ad.X)
            clustering_labels = kmeans.labels_

            ## ContrastiveVI end
            end_time = time.perf_counter()


            ari = adjusted_rand_score(true_labels, clustering_labels)
            print(ari)

            from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

            accuracy = accuracy_score(true_labels, clustering_labels)
            precision = precision_score(true_labels, clustering_labels)
            recall = recall_score(true_labels, clustering_labels)
            f1 = f1_score(true_labels, clustering_labels)

            print("Accuracy:", accuracy)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1-score:", f1)

            distances = kmeans.transform(salient_ad.X)
            y_true = true_labels
            y_scores = distances[:, 0] - distances[:, 1]

            from sklearn.metrics import roc_auc_score, average_precision_score
            from sklearn.metrics import roc_curve, precision_recall_curve

            # Calculate AUC
            auc = roc_auc_score(y_true, y_scores)
            print("AUC:", auc)

            # Calculate AUPR
            aupr = average_precision_score(y_true, y_scores)
            print("AUPR:", aupr)

            salient_ad_all = AnnData(
                X=model.get_latent_representation(ad, representation_kind='salient'),
                obs=ad.obs
            )
            sc.pp.pca(salient_ad_all)

            X = salient_ad_all.X

            val = np.sum(np.abs(X), axis=1)

            val_ref = val[ad.obs["dataset"].isin(['reference'])]
            val_que = val[ad.obs["dataset"].isin(['query'])]
            ad_que = ad[ad.obs["dataset"].isin(['query'])]
            val_que_healthy = val_que[ad_que.obs['final_cluster'].isin(healthy_celltypes)]
            val_que_disease = val_que[ad_que.obs['final_cluster'].isin(disease_celltypes)]

            thres = np.percentile(val_ref, 95)
            predicted_labels = (val_que > thres).astype(int)
            true_labels = ad_que.obs['final_cluster'].isin(disease_celltypes).to_numpy().astype(int)

            from sklearn.metrics import adjusted_rand_score
            ari = adjusted_rand_score(true_labels, predicted_labels)

            accuracy1 = accuracy_score(true_labels, predicted_labels)
            precision1 = precision_score(true_labels, predicted_labels)
            recall1 = recall_score(true_labels, predicted_labels)
            f11 = f1_score(true_labels, predicted_labels)

            y_scores = val_que

            # Calculate AUC
            auc1 = roc_auc_score(y_true, y_scores)
            print("AUC:", auc)

            # Calculate AUPR
            aupr1 = average_precision_score(y_true, y_scores)
            print("AUPR:", aupr)


            time_taken = end_time - start_time

            print(PROBLEM_KEY, max(accuracy, accuracy1), max(precision, precision1), max(recall1, recall), max(f1, f11), max(auc, auc1), max(aupr, aupr1), time_taken,  sep=',', file=f, flush=True)


<ipython-input-5-ac08c00d6e58>:16: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  ad.layers['counts'] = ad.X.copy()
<ipython-input-7-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-5-ac08c00d6e58>:16: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  ad.layers['counts'] = ad.X.copy()
<ipython-input-7-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-5-ac08c00d6e58>:16: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  ad.layers['counts'] = ad.X.copy()
<ipython-input-7-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.o

RUN:  0


INFO:pytorch_lightning.utilities.seed:Global seed set to 0
INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/usr/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


AnnData object with n_obs × n_vars = 2329 × 15970
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'integrated_snn_res.0.2', 'seurat_clusters', 'integrated_snn_res.0.6', 'original', 'sample', 'integrated_snn_res.1.2', 'final_cluster', 'orig_id', 'sample_id', 'neworig', 'orig_id2', 'percent.mt', 'total', 'obs_names', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'dataset', 'batch_key', 'zone', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    layers: 'counts'
AnnData object with n_obs × n_vars = 3304 × 15970
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'integrated_snn_res.0.2', 'seurat_clusters', 'integrated_snn_res.0.6', 'original', 'sample', 'integrated_snn_res.1.2', 'final_cluster', 'orig_id', 'sample_id', 'neworig', 'orig_id2', 'percent.mt', 'total', 'obs_names', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'dataset', 'batch_key', 'zone', '_scvi_batch', '_scvi_labels'

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/lightning.py:2066: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook
INFO:pytorch_lightning.utilities.distributed:GPU available: False, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


...

Epoch 155/500:  31%|███       | 155/500 [06:12<13:48,  2.40s/it, loss=3.31e+03, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 3445.885. Signaling Trainer to stop.
INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        
0.16780080680747497
Accuracy: 0.2949891067538126
Precision: 0.330719794344473
Recall: 0.4715909090909091
F1-score: 0.3887881535207011
AUC: 0.2219830568303013
AUPR: 0.3294001300660387


INFO:pytorch_lightning.utilities.seed:Global seed set to 400


AUC: 0.2219830568303013
AUPR: 0.3294001300660387
RUN:  4
AnnData object with n_obs × n_vars = 11475 × 1426
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'integrated_snn_res.0.2', 'seurat_clusters', 'integrated_snn_res.0.6', 'original', 'sample', 'integrated_snn_res.1.2', 'final_cluster', 'orig_id', 'sample_id', 'neworig', 'orig_id2', 'percent.mt', 'total', 'obs_names', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'dataset', 'batch_key', 'zone', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    layers: 'counts'
AnnData object with n_obs × n_vars = 3304 × 1426
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'integrated_snn_res.0.2', 'seurat_clusters', 'integrated_snn_res.0.6', 'original', 'sample', 'integrated_snn_res.1.2', 'final_cluster', 'orig_id', 'sample_id', 'neworig', 'orig_id2', 'percent.mt', 'total', 'obs_names', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'data

INFO:pytorch_lightning.utilities.distributed:GPU available: False, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


Epoch 160/500:  32%|███▏      | 160/500 [06:24<13:36,  2.40s/it, loss=3.3e+03, v_num=1]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 3442.878. Signaling Trainer to stop.
INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        
0.17165170932392515
Accuracy: 0.7073638344226579
Precision: 0.7941671340437465
Recall: 0.5190615835777126
F1-score: 0.6277987142540457
AUC: 0.8162473434563764
AUPR: 0.8104651148115496
AUC: 0.8162473434563764
AUPR: 0.8104651148115496


In [10]:
metrics = pd.read_csv('metrics.csv')

In [11]:
metrics

,PROBLEM_KEY,accuracy,precision,recall,f1,auc,aupr,time_taken
0,ref_0_que_1hr,0.795832,0.687121,0.846082,0.758361,0.884597,0.813002,1370.832391
1,ref_0_que_1hr,0.620276,0.454545,0.127799,0.103670,0.680312,0.514057,1147.225773
2,ref_0_que_1hr,0.847757,0.751767,0.892724,0.816205,0.926816,0.876651,1078.863644
3,ref_0_que_1hr,0.829036,0.738250,0.849813,0.790113,0.903602,0.844047,1201.941569
4,ref_0_que_1hr,0.640057,0.521423,0.601679,0.558683,0.694605,0.587954,1068.539259
5,ref_0_que_4hr,0.889640,0.926768,0.891859,0.908978,0.955604,0.967821,1222.299536
6,ref_0_que_4hr,0.859610,0.938017,0.827461,0.879277,0.946995,0.963528,838.751884
7,ref_0_que_4hr,0.464715,0.593537,0.424058,0.494685,0.466614,0.610590,954.361929
8,ref_0_que_4hr,0.879880,0.918033,0.884569,0.900990,0.938793,0.952344,1059.375420
9,ref_0_que_4hr,0.497748,0.591232,0.606318,0.598680,0.435727,0.568812,1011.479216


In [12]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t

# Confidence level
confidence = 0.95

# Function to calculate mean ± margin
def mean_margin(series, confidence=0.95):
    n = len(series)
    mean = np.mean(series)
    se = sem(series)
    margin = se * t.ppf((1 + confidence) / 2., n - 1)
    return f"{mean:.4f} ± {margin:.4f}"

# Group by 'Group' and calculate mean ± margin for other columns
grouped = metrics.groupby('PROBLEM_KEY')

result = grouped.agg({
    col: lambda x: mean_margin(x, confidence)
    for col in metrics.columns if col != 'PROBLEM_KEY'
})

# Reset index to make it look cleaner
result.reset_index(inplace=True)

In [13]:
result

,PROBLEM_KEY,accuracy,precision,recall,f1,auc,aupr,time_taken
0,ref_0_que_1,0.6844 ± 0.1534,0.8170 ± 0.1492,0.5207 ± 0.2218,0.6289 ± 0.2099,0.7609 ± 0.1852,0.8098 ± 0.1473,258.7683 ± 34.2012
1,ref_0_que_1-7,0.6736 ± 0.0540,0.6751 ± 0.1336,0.7728 ± 0.0945,0.6724 ± 0.0259,0.7686 ± 0.0578,0.7626 ± 0.0581,402.1787 ± 73.7155
2,ref_0_que_1hr,0.7466 ± 0.1343,0.6306 ± 0.1670,0.6636 ± 0.3982,0.6054 ± 0.3704,0.8180 ± 0.1492,0.7271 ± 0.2042,1173.4805 ± 152.5935
3,ref_0_que_3,0.7109 ± 0.0862,0.6649 ± 0.1037,0.5896 ± 0.2448,0.5896 ± 0.1768,0.7614 ± 0.1320,0.6784 ± 0.1447,945.0769 ± 158.4111
4,ref_0_que_4hr,0.7183 ± 0.2695,0.7935 ± 0.2282,0.7269 ± 0.2548,0.7565 ± 0.2426,0.7487 ± 0.3376,0.8126 ± 0.2534,1017.2536 ± 175.2349
5,ref_0_que_7,0.6241 ± 0.0229,0.6511 ± 0.0753,0.6398 ± 0.2760,0.5808 ± 0.1392,0.6893 ± 0.0104,0.6243 ± 0.0087,918.1800 ± 244.6379


In [14]:
result.to_csv('result.csv', index=False)